In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()

In [3]:
conf.setMaster('local[*]')

In [7]:
conf.setAppName('sandbox')

In [ ]:
sc = SparkContext(conf=conf)

In [9]:
print(sc)

<SparkContext master=local[*] appName=sandbox>


In [10]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addFile',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'binaryRecords',
 'broadcast',
 'cancelAllJobs',
 '

In [12]:
sc.textFile()

TypeError: textFile() missing 1 required positional argument: 'name'

In [45]:
tags_df = sc.textFile('tags.csv')
ratings_df = sc.textFile('./ratings.csv')
movies_df = sc.textFile('./movies.csv')

In [17]:
tags_df

tags.csv MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [19]:
dir(tags_df)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_jrdd',
 '_jrdd_deserializer',
 '_memory_limit',
 '_pickled',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'flatMapValues',
 'fold',
 'foldByKey',
 'foreach',
 'foreachPartition',
 'fullOuterJoin',
 'getCheckpointFile',
 'getNumPartitions

In [22]:
tags_df.take(5)

['userId,movieId,tag,timestamp',
 "15,339,sandra 'boring' bullock,1138537770",
 '15,1955,dentist,1193435061',
 '15,7478,Cambodia,1170560997',
 '15,32892,Russian,1170626366']

In [65]:
from itertools import starmap

tags_dep = tags_df.map(lambda line: line.split(','))
ratings_dep = ratings_df.map(lambda line: line.split(','))
movies_dep = movies_df.map(lambda line: line.split(','))

In [47]:
tags_dep.take(10)

[['userId', 'movieId', 'tag', 'timestamp'],
 ['15', '339', "sandra 'boring' bullock", '1138537770'],
 ['15', '1955', 'dentist', '1193435061'],
 ['15', '7478', 'Cambodia', '1170560997'],
 ['15', '32892', 'Russian', '1170626366'],
 ['15', '34162', 'forgettable', '1141391765'],
 ['15', '35957', 'short', '1141391873'],
 ['15', '37729', 'dull story', '1141391806'],
 ['15', '45950', 'powerpoint', '1169616291'],
 ['15', '100365', 'activist', '1425876220']]

In [48]:
ratings_dep.take(10)

[['userId', 'movieId', 'rating', 'timestamp'],
 ['1', '31', '2.5', '1260759144'],
 ['1', '1029', '3.0', '1260759179'],
 ['1', '1061', '3.0', '1260759182'],
 ['1', '1129', '2.0', '1260759185'],
 ['1', '1172', '4.0', '1260759205'],
 ['1', '1263', '2.0', '1260759151'],
 ['1', '1287', '2.0', '1260759187'],
 ['1', '1293', '2.0', '1260759148'],
 ['1', '1339', '3.5', '1260759125']]

In [68]:
movies_df.take(10)

['movieId,title,genres',
 '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy',
 '6,Heat (1995),Action|Crime|Thriller',
 '7,Sabrina (1995),Comedy|Romance',
 '8,Tom and Huck (1995),Adventure|Children',
 '9,Sudden Death (1995),Action']

In [111]:
movies_header = movies_df.first()
movies_rdd = movies_df.filter(lambda line: line != movies_header)
# movies_rdd = movies_df.map(lambda x: int(x[0]))
movies_rdd = movies_rdd.map(lambda line: line.split(',')).map(lambda x: (int(x[0]), (x[1], set(x[2].split('|')))))

In [114]:
src_movie = (31, ('Dangerous Minds (1995)', {'Drama'}))
#movies_rdd.filter(lambda x: x[0] == a).first()
movies_rdd.filter(lambda movie: movie[1][1].intersection(src_movie[1][1])).take(10)

[(4, ('Waiting to Exhale (1995)', {'Comedy', 'Drama', 'Romance'})),
 (14, ('Nixon (1995)', {'Drama'})),
 (16, ('Casino (1995)', {'Crime', 'Drama'})),
 (17, ('Sense and Sensibility (1995)', {'Drama', 'Romance'})),
 (20,
  ('Money Train (1995)', {'Action', 'Comedy', 'Crime', 'Drama', 'Thriller'})),
 (22, ('Copycat (1995)', {'Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'})),
 (24, ('Powder (1995)', {'Drama', 'Sci-Fi'})),
 (25, ('Leaving Las Vegas (1995)', {'Drama', 'Romance'})),
 (26, ('Othello (1995)', {'Drama'})),
 (27, ('Now and Then (1995)', {'Children', 'Drama'}))]